PART 2: Optimized Deep Learning Model

In [20]:
# Import the needed dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv from external webpage.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
# Drop the non-beneficial ID columns.
# This time DON'T drop "NAME" column as we want it to improve model's accuracy.
application_df = application_df.drop(["EIN"], axis=1)

In [22]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [23]:
# Look at "NAME" value counts for binning.
name_counts = application_df['NAME'].value_counts()
# Find how many name counts are greater than 5.
name_counts[name_counts>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [24]:
# Determine which values to replace if counts are less than or equal to 5.
name_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe.
for app in name_replace:
    application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful.
application_df['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [27]:
# Look at APPLICATION_TYPE value counts for binning.
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [28]:
# We choose 500 as our cutoff for number of application counts.
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful.
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [29]:
# Look at CLASSIFICATION value counts for binning.
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [30]:
# Determine which values to replace if counts are less than 1000.
classes_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe.
for cls in classes_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful.
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [31]:
# Generate categorical variable lists.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [32]:
# Convert categorical data to numeric with `pd.get_dummies`.
application_dummies_df = pd.get_dummies(application_df)

In [33]:
# Split preprocessed data into independent variables and dependent variable arrays.
X = application_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_dummies_df["IS_SUCCESSFUL"].values

# Use random state of 42 so that model produces same results every time that it runs.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [34]:
# Create StandardScaler instance.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])# number of input features determined by number of columns in X_train variable
hidden_layer1 =  100
hidden_layer2 = 30
hidden_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_layer1, input_dim=number_input_features, activation="relu")
)# use 'relu' for first hidden layer

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="sigmoid"))# use sigmoid for second hidden layer

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation="sigmoid"))# use sigmoid for third hidden layer

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))# use sigmodi for output layer
# Check the structure of the model.
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               39900     
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 10)                310       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43251 (168.95 KB)
Trainable params: 43251 (168.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model.
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model.
# Run model through 100 epochs.
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.4911 - accuracy: 0.7747
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4417 - accuracy: 0.7936
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4340 - accuracy: 0.7968
Epoch 4/100
804/804 [==============================] - 4s 4ms/step - loss: 0.4296 - accuracy: 0.7980
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4259 - accuracy: 0.7983
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4242 - accuracy: 0.7986
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4224 - accuracy: 0.7991
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4210 - accuracy: 0.7990
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4198 - accuracy: 0.7996
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4190 - accura

In [38]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4746 - accuracy: 0.7836 - 576ms/epoch - 2ms/step
Loss: 0.4746280312538147, Accuracy: 0.7835568785667419


In [39]:
# Export optimized model to HDF5 file.
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### Data Preprocessing
----
1. What variable(s) are considered the target for your model?
ANSWER: "Is-Successful" column.
---
2. What variable(s) are considered to be the features for your model?
ANSWER: NAME, APPLICATION, TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, INCOME_AMT,SPECIAL_CONSIDERATIONS, STATUS, and ASK_AMT
---
3. What variable(s) are neither and should be removed from the input data?
ANSWER:  EIN, SPECIAL_CONSIDERATIONS, STATUS


#### Compiling, Training, and Evaluating the Model
----
1. How many neurons, layers, and and activation functions did you select for your neural network model, and why?
ANSWER: Three hidden layers.  Changing the 2nd and 3rd activation functions to 'sigmoid' also helped boost the accuracy.
---
2. Were you able to achieve the target model performance?
ANSWER: Yes
---
3. What steps did you take to try and increase model performance?
ANSWER: Converted the NAME column into data points.  Also added a third layer and used "sigmoid" activation function for the 2nd and 3rd layer.